In [1]:
import pandas as pd
import requests
import numpy as np
import time
from tqdm.notebook import tqdm
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
api_key = ""

In [3]:
collections_dict = {}

In [7]:
def collection_info(collection_contract,api_key):
    headers = {"accept": "application/json", "X-API-Key":api_key}
    url = f"https://deep-index.moralis.io/api/v2/nft/{collection_contract}/trades?chain=eth&marketplace=opensea" # &offset=500&limit=1000
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        print(f"API returned status code {response.status_code}")
        print(f"Probably you have exceeded API limitations!")
        return
    content = response.json()
    limit = int(content["total"])
    total_pages = int(np.ceil(limit / 500))
    buyers = []
    sellers = []
    assets = []
    prices_in_eth = []
    block_timestamps = []
    payment_token = []
    for i in range(total_pages):
        try:
            offset = i*500
            url = f"https://deep-index.moralis.io/api/v2/nft/{collection_contract}/trades?chain=eth&marketplace=opensea&offset={offset}&limit={limit}"
            response = requests.request("GET", url, headers=headers)
            time.sleep(2)# else API rate breaks easily
            content = response.json()
            for tr in content["result"]:
                for asset in tr["token_ids"]:
                    assets.append(asset)
                    buyers.append(tr["buyer_address"])
                    sellers.append(tr["seller_address"])
                    prices_in_eth.append(float(tr["price"])/len(tr["token_ids"])) #decimals of eth
                    payment_token.append(tr["price_token_address"])
                    block_timestamps.append(tr["block_timestamp"])
        except KeyError:
            continue
    collection_addreses = pd.DataFrame(np.array([buyers,sellers,assets,prices_in_eth,block_timestamps,payment_token]).T, columns=["buyer","seller","asset_id","price_in_eth","block_timestamp","payment_token"])
    collection_addreses["block_timestamp"] = pd.to_datetime(collection_addreses["block_timestamp"])
    collection_addreses["collection"] = collection_contract
    return collection_addreses

In [8]:
collections = pd.read_csv("./data/opensea_collections.csv",index_col=0)

In [9]:
len(collections)

824

In [10]:
## Extremely slow process
for collection in tqdm(collections["collection"].tolist()):# keep adding when error
    collection_info_df = collection_info(collection,api_key)
    collections_dict[collection] = collection_info_df

  0%|          | 0/824 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [13]:
len(collections_dict)

54

In [14]:
with open("./data/collections_dict.pickle", "wb") as f:
    pickle.dump(collections_dict,f)

In [ ]:
coll_txns_list = []
for el in collections_dict:
    coll_txns_list.append([collections_dict[el].shape[0],el])

In [ ]:
sorted_colls = pd.DataFrame(coll_txns_list,columns=["txns_num","address"]).sort_values(by="txns_num",ascending=False).reset_index(drop=True)

In [ ]:
sorted_colls["txns_num"].plot(kind="hist");

In [ ]:
sorted_colls[sorted_colls["txns_num"] >=10000].head(10)

In [ ]:
data

In [ ]:
def plot_stats(collection, slow_plot=False):
    data = collections_dict[collection].copy()
    data["price_in_eth"] = data["price_in_eth"].astype(float)
    data["block_timestamp"] = pd.to_datetime(data["block_timestamp"])
    data["acc_price"] = data["price_in_eth"].cumsum()
    
    if slow_plot:
        sns.lineplot(x="block_timestamp",y="acc_price",data=data);
    
    daily_grouped_data = data[["block_timestamp","price_in_eth"]].groupby(pd.Grouper(key='block_timestamp', axis=0, 
                          freq='d')).agg(["mean","count"])
    daily_grouped_data = daily_grouped_data.reset_index()
    daily_grouped_data.columns = ["block_timestamp", "avg_d_price", "count_trades"]
    days = (data["block_timestamp"].max()-data["block_timestamp"].min()).days
    num_assets = len(set(data["asset_id"].tolist()))
    print(f"For collection {collection}")
    print(f"Total trading days is: {days}")
    print(f"Total traded assets: {num_assets}")
    sns.lineplot(x="block_timestamp",y="avg_d_price",data=daily_grouped_data);
    plt.show()
    
    sns.lineplot(x="block_timestamp",y="count_trades",data=daily_grouped_data);
    plt.show()
    
    daily_grouped_data["avg_d_price"].plot(kind="box");
    plt.show()
    
    daily_grouped_data["count_trades"].plot(kind="box");
    plt.show()

In [ ]:
for collection in sorted_colls[sorted_colls["txns_num"] >=10000][10:].head(10)["address"].tolist():
    plot_stats(collection)